Code by Marvin Kinz, m.kinz@stud.uni-heidelberg.de
# Chamfer distance

## Calculate Chamfer distances
For undeformed to deformed objects and slices and for deformed cut sides on neighboring slices, that belong together.


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import glob
from pathlib import Path 

def Chamfer(folder,defs,type,show=0):
    """
    Function to calculate chamfer distances for undeformed to deformed objects and slices and for deformed cut sides on neighboring slices, that belong together.

    params
    ------
    folder : string
        Folder where the different gravities are placed in
    defs : array shape (N)
        Array of N gravity values
    type : string [Objects, Slices, Cuts]
        Type of data to calculate chamfer distances for
    show : bool
        Set true if you want a visualization of the process.
    

    returns
    -------
    chamf : array shape (Number of files, N)
        Calculated chamfer distances get returned and saved as textfile in folder.

    """
    #Determine files to be loaded depending on the type
    files=[]
    if type=="Slices":
        undeformed="Slices"
        deformed="Slices Deformed"
        for obj in glob.glob(f"{folder}/Gravity_{defs[0]}/{undeformed}/*.xyz"):
            files.append(Path(obj).stem)
    elif type == "Objects":
        undeformed="Initial"
        deformed="Deformed"
        for obj in glob.glob(f"{folder}/{undeformed}/*.xyz"):
            files.append(Path(obj).stem)
    elif type == "Cuts":
        undeformed="CameraCaptures"
        deformed=undeformed
        for obj in glob.glob(f"{folder}/Gravity_{defs[0]}/{undeformed}/*.xyz"):
            files.append(Path(obj).stem)
    if type=="Cuts":
        #Files and matching files are the border points, which will be used for initial alignement. The sampled once will then be used for calculating the chamfer distance
        filessampled=files[0::4]
        matchingfilessampled=files[2::4]
        matchingfiles=files[3::4]
        files=files[1::4]
    #Set up result array
    chamf=np.zeros((len(files),len(defs)))
    #Go through all files and in case of objects load initial
    for c,obj in enumerate(files):
        if type == "Objects": def0=np.loadtxt(f"{folder}/{undeformed}/{obj}.xyz")
        #Go through all matching files and load initial
        for i,d in enumerate(defs):
            if not type == "Objects": def0=np.loadtxt(f"{folder}/Gravity_{d}/{undeformed}/{obj}.xyz")
            
            #Least squares fitting https://gist.github.com/scturtle/c3037529098338eccc403a9842870273
            p1init,ind=np.unique(def0,return_index=True,axis=0)
            p1=p1init.T
            p1c = np.mean(p1, axis=1).reshape(-1, 1)
            q1 = p1 - p1c


            #set point cloud
            pcd1 = o3d.geometry.PointCloud()
            if type=="Cuts":
                p1sampled=np.loadtxt(f"{folder}/Gravity_{d}/{undeformed}/{filessampled[c]}.xyz")
                pcd1.points = o3d.utility.Vector3dVector(p1sampled)
            else:            
                pcd1.points = o3d.utility.Vector3dVector(p1init)

            #Load deformed and fit 
            if type=="Cuts":
                def2=np.loadtxt(f"{folder}/Gravity_{d}/{deformed}/{matchingfiles[c]}.xyz")
                p2sampled=np.loadtxt(f"{folder}/Gravity_{d}/{undeformed}/{matchingfilessampled[c]}.xyz")
            else:
                def2=np.loadtxt(f"{folder}/Gravity_{d}/{deformed}/{obj}.xyz")
            #Check for incompatible point clouds
            if def2.shape!=def0.shape: 
                chamf[c,i]= np.nan
                continue

            p2=def2[ind].T
            p2c = np.mean(p2, axis=1).reshape(-1, 1)
            q2 = p2 - p2c
            H = sum([q1[:, i].reshape(-1, 1).dot(q2[:, i].reshape(1, -1))
                    for i in range(q1.shape[1])])
            U, _, V = np.linalg.svd(H)
            R2 = V.T.dot(U.T)
            T2 = p2c - R2.dot(p1c)
        
            #Set point clouds
            pcd2 = o3d.geometry.PointCloud()
            pcd3 = o3d.geometry.PointCloud()
            if type=="Cuts":
                pcd2.points = o3d.utility.Vector3dVector(p2sampled)
                p3 = R2.dot(p1sampled.T) + T2
                pcd3.points = o3d.utility.Vector3dVector(p3.T)
            else:
                pcd2.points = o3d.utility.Vector3dVector(p2.T)
                p3 = R2.dot(p1) + T2
                pcd3.points = o3d.utility.Vector3dVector(p3.T)
                

            #Calculate Chamfer Distance
            dist1 = np.asarray(pcd3.compute_point_cloud_distance(pcd2))
            dist2 = np.asarray(pcd2.compute_point_cloud_distance(pcd3))
            chamf[c,i]=np.sum(dist1**2)/len(dist1)+np.sum(dist2**2)/len(dist2)
            if show: #for only showing a special match: and i>6 and c>6:
                pcd1.paint_uniform_color([1,0,0])
                pcd2.paint_uniform_color([0,1,0])
                pcd3.paint_uniform_color([0,0,1])
                o3d.visualization.draw_geometries([pcd1,pcd2,pcd3])
    np.savetxt(f"{folder}/Chamfer_{type}.txt",chamf)
    return chamf

In [8]:
def plot_chamfer(chamf, defs, object, num):
    """
    Function to plot chamfer distances.

    params
    ------
    chamf : array shape (Number of files, N)
        Calculated chamfer distances
    defs : array shape (N)
        Array of N gravity values
    object : string 
        Name of objects, gets used in title and filename of the resulting plot
    num : int [1,inf)
        In case of slices or cuts set number here to get chamfer distances for each plotted
    

    returns
    -------
        Plot of chamfer distances stored in Plots subfolder
    """
    
    plt.plot(defs,np.nanmean(chamf,axis=0),"-x",label="All Objects")
    if num>1:
        for i in range(num):
            plt.plot(defs,np.nanmean(chamf[i::num],axis=0),"-x",label=f"Object {i}")
    plt.title(f"Chamfer distance as function of gravity \n for {object} objects")
    plt.ylabel("Chamfer distance")
    plt.xlabel("Gravity [m/s^2]")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"Plots/Chamfer {object}.pdf", dpi=400)

In [ ]:
#Example for chamfer distance of cuts with plotting
defs=np.arange(0,4100,500)
Chamf=Chamfer("2 Deformation and Slicing/SimulationResults/DeformationCube (SameGravityAfterCut)",defs,"Objects")
plot_chamfer(Chamf,defs,"Cube",2)

In [ ]:
#Example how to calculate chamfer distances for multiple objects
folderOcta="2 Deformation and Slicing/SimulationResults/DeformationOcta (SameGravityAfterCut)"
folderCube="2 Deformation and Slicing/SimulationResults/DeformationCube (SameGravityAfterCut)"
folderCone="2 Deformation and Slicing/SimulationResults/DeformationCone (SameGravityAfterCut)"

defs=np.arange(0,4100,500)
ChamfCube=Chamfer(folderCube,defs,"Objects")
ChamfCone=Chamfer(folderCone,defs,"Objects")
ChamfOcta=Chamfer(folderOcta,defs,"Objects")

In [ ]:
#Example how to show different objects in one plot
plt.plot(defs,np.nanmean(ChamfCube,axis=0),"-x",label="Cube")
plt.plot(defs,np.nanmean(ChamfCone,axis=0),"-x",label="Cone")
plt.plot(defs,np.nanmean(ChamfOcta,axis=0),"-x",label="Octahedron")
plt.title(f"Chamfer distance as function of gravity for different objects")
plt.ylabel("Chamfer distance")
plt.xlabel("Gravity [m/s^2]")
plt.legend()
plt.tight_layout()
plt.savefig(f"Plots/Chamfer Distance Objects.pdf", dpi=400)